---

# balanced turbulence with internal waves configuration

The cpp key is ITBALT

The type of jet (1 to 4) is defined in `cppdefs.h`

Run time options are found in `croco.in`

## commands from start to finish:

```
./jobcomp
python chain_datarmor.py jet_cfg1_wp75_4km_0a2000j 20 03:00:00 4 jetn 0
# and release
```

Restart:
```
python chain_datarmor.py jet_cfg1_wp75_4km_nodecay_2000a3000j 10 03:00:00 4 jetn 1
cp /home2/pharos/othr/aponte/croco/jet_cfg1_wp75_4km_0a2000j/t20/jetn_rst*.nc /home1/scratch/aponte/jet_cfg1_wp75_4km_nodecay_2000a3000j/t0/
# and release
```

## outputs:

see [`iodef.xml`](iodef.xml)

### file_his_*.nc

Relevant temporal variable: `time_counter`

Variables are 2D or 3D:

- `v_a` : temporal averaged of variable $v$

- `v_t_cos` : temporal averaged of $v\times\cos(\omega t)$

- `v_t_sin` : temporal averaged of $v\times\cos(\omega t)$

- `v_t_dirac` : instantaneous value of $v$ at the center of the interval

The relevant time interval for this output has size `freq_op` (2d typically) and is outputed every `output_freq` (25d typically).

![his](../JETN/croco_jetn_time.001.png)


### file_ave_*.nc

Relevant temporal variable: `time_counter`

Variables are 2D and averaged over a temporal window of size `output_freq` (2d typically).

- `v` : temporal averaged of variable $v$

- `v_t_cos` : temporal averaged of $v\times\cos(\omega t)$

- `v_t_sin` : temporal averaged of $v\times\cos(\omega t)$

![ave](../JETN/croco_jetn_time.002.png)

### file_surf_*.nc

Relevant temporal variable: `time_instant`

Variables are 2D (surface) and instantaneous every `output_freq` (10 min typically).

### file_inst_*.nc, file_sta1_*.nc, ...

Relevant temporal variable: `time_instant`

Variables are 1D and instantaneous every `output_freq` (30min typically).

### file_swath_*.nc

Not that useful for now.


### nicer treatment of time coordinate

Work is ongoing around xarray in order to implement useful features for this
such as selection alon non-dim coordinates [issue](https://github.com/pydata/xarray/issues/1603)


FORCAGE STOCHASTIQUE DE LA TURBULENCE
===

Ce développement implémente un forçage stochastique de la turbulence. Ce forçage est calculé tous les *dtfst* pas de temps (en seconde) et est interpolé entre ces deux calculs consécutifs. 


La fonction de forçage est déninie de la façon suivante:

$$
\begin{align}
F_{n\Delta t}(i,j,k) = F_{T}.W(i,j).F^{-1}[\sum_{n} N_{n\Delta t}(kx,ky,n).\psi(kx,ky,n).\phi_{n}(k)]
\end{align}
$$

$ F_{T} $ est une constante (qui se déclinera ensuite en $ F_{u}, F_{v} $ ) pour forcer la densité

$ W(i,j) $ est une fonction de forme du forçage (pour l'instant seulement en y) dans l'espace physique

$$ W(i,j) = e^{-((yr(i,j)-y_{F})/(\Delta y_{F}))^{2}} $$

$N_{n\Delta t}(kx,ky,n)$ est un bruit blanc. C'est une fonction aléatoire reproductible dans le temps.

$ \psi(kx,ky,n) $ définit l'amplitude du signal

$$ 
\psi (kx,ky,n)= \underbrace{$a_{n}}_{mode} \underbrace{e^{-(kx^{2} + ky^{2} - kF^{2})/(\Delta kF^{2}) }}_{horizontal}
$$

$\phi_{n}$ sont les modes verticaux

Implémentation dans le code
===

- cppdefs.h : ajout d'une clé *FSTURB* pour définir le forçage
- ana_fsturb.F : nouveau module qui regroupe les fonctions d'initialisation et de calcul du forçage dans le temps
- main.F : initialisation du forçage ( *call init_fsturb*)
- step.F : calcul d'un nouveau forçage tous les $\Delta t$ pas de temps
- step3d_t.F : ajout du forçage interpolé en temps et en z
 
Paramètres
===

- *Nmodefst* : Nombre de modes
- dtfst = 1.*86400
- *afst* : $a_{n}$ amplitude des différents modes
- *xfstmid* :
- *xfstwid* :
- *yfstmid* : $ y_{F} $
- *yxfstwid* : $ \Delta y_{F} $
- *FTfst* : $ F_{T} $
- *kFfst* : $ kF $
- *dkFfst* : $ \Delta kF $

Variables
===

Toutes les variables relatives au forçage sont déclarées dans le module *fsturb*, dans le fichier *ana_fsturb.F*

- *Ffst(i,j,k,t)* : $ F_{n\Delta t}(i,j,k) $
- *FTfst* : $ F_{T} $ défini en parameter dans le module
- *Wfst* : W(i,j)
- *Noisefst* : $ N_{n\Delta t}(kx,ky,n) $ 
- *afst* : $a_{n}$ amplitude des différents modes
- *psifst* : $\psi(kx,ky,n)$
- *phir* : modes calculés dans vmodes
